In [2]:
import pandas as pd
import numpy as np
import random
import csv

In [3]:
recipe_data = pd.read_csv("../recipe.csv")
recipe_data.fillna("UNK", inplace=True)

cuisine_data = pd.read_csv("../cooking_data(cuisine).csv", sep=';')

chef_data = pd.read_csv("../chef.csv")

In [58]:
recipe_data.head()

,type,difficulty,name,description,prep_time,cook_time,cuisine_name,ingredient_name,portions,image_id
0,meal,3,Tavë Kosi,A traditional Albanian dish of baked lamb and ...,20,60,Albanian,lamb,1,0
1,dessert,4,Bakllasarem,A sweet pastry filled with a mixture of walnut...,30,45,Albanian,phyllo dough,1,0
2,meal,2,Fërgesë,"A hearty dish of peppers, tomatoes, and cottag...",15,30,Albanian,red bell pepper,1,0
3,meal,3,Speca të mbushur,"Bell peppers stuffed with rice, herbs, and sea...",20,40,Albanian,red bell pepper,1,0
4,meal,3,Byrek me Spinaq,"A savory pie made with phyllo dough, filled wi...",20,30,Albanian,phyllo dough,1,0


In [5]:
# Cuisines generator

cuisines = {
    "name":[],
    "image_id":[]
}

for row in cuisine_data.itertuples(index=True, name='Pandas'):
    cuisines["name"].append(getattr(row, "name"))
    if(len(cuisines["image_id"]) != 0):
        cuisines["image_id"].append(cuisines["image_id"][-1]+1)
    else:
        cuisines["image_id"].append(1)
    
for row in recipe_data.itertuples(index=True, name='Pandas'):
    cuisine_name = getattr(row, "cuisine_name")
    if not(cuisine_name in cuisines["name"]):
        cuisines["name"].append(cuisine_name)
        cuisines["image_id"].append(cuisines["image_id"][-1]+1)

print(cuisines)
pd.DataFrame(cuisines).to_csv("cuisine.csv", index=False, quoting=csv.QUOTE_NONNUMERIC, quotechar='"')

{'name': ['Greek', 'French', 'Indian', 'Italian', 'Japanese', 'Chinese', 'Mexican', 'German', 'Korean', 'Turkish', 'Albanian', 'American', 'Bulgarian', 'Russian'], 'image_id': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]}


In [60]:
# Modify recipes

# Change column names
new_recipe = recipe_data.rename(columns={'cook_time': 'cook_time_m', 'prep_time': 'prep_time_m'})

# Change type names
new_recipe['type'] = np.where(new_recipe['type'] == 'meal', 'Cooking', 'Pastry')

# Change cuisine names to include all cuisines
rec_len = len(new_recipe['cuisine_name'])
new_cuisine_name = { 'cuisine_name': [] }

for i in range(0, rec_len):
    new_cuisine_name['cuisine_name'].append(cuisines['name'][i % len(cuisines['name'])])

new_recipe.update(pd.DataFrame(new_cuisine_name))
new_recipe.to_csv("recipe.csv", index=False, quoting=csv.QUOTE_NONNUMERIC, quotechar='"')

In [6]:
# chef_has_cuisine generator

chef_has_cuisine = {
    "chef_id":[],
    "cuisine_name":[]
}

chef_ids = []
for row in chef_data.itertuples(index=True, name='Pandas'):
    chef_ids.append(getattr(row, 'id'))

for id in chef_ids:
    s = random.randint(1, 3)
    selection = random.sample(cuisines["name"], s)
    
    for i in range(0, s):
        chef_has_cuisine["chef_id"].append(id)       
        chef_has_cuisine["cuisine_name"].append(selection[i])

pd.DataFrame(chef_has_cuisine).to_csv("chef_has_cuisine.csv", index=False, quoting=csv.QUOTE_NONNUMERIC, quotechar='"')

In [7]:
nr_data = pd.read_csv("recipe.csv")
recipe_cuisine = {}
for row in nr_data.itertuples(index=True, name='Pandas'):
    recipe_cuisine[getattr(row, 'name')] = getattr(row, 'cuisine_name')

In [16]:
# recipe_has_chef generator

recipe_has_chef = {
    "recipe_name": [],
    "recipe_cuisine_name": [],
    "chef_id": []
}

# key: cuisine, value: list of chef ids that has that cuisine
rcuisine = {}
for i in range(0, len(chef_has_cuisine["chef_id"])):
    c = chef_has_cuisine["cuisine_name"][i]
    id = chef_has_cuisine["chef_id"][i]
    if not(c in rcuisine.keys()):
        rcuisine[c] = [id]
    else:
        rcuisine[c].append(id)
#print(rcuisine)

# key: cuisine, value: list of recipes that are of this cuisine
rrecipe = {}
for lr in recipe_cuisine.keys():
    lc = recipe_cuisine[lr]
    if not(lc in rrecipe.keys()):
        rrecipe[lc] = [lr]
    else:
        rrecipe[lc].append(lr)
#print(rrecipe)

for cuisine in rcuisine.keys():
    for id in rcuisine[cuisine]:
        for rec in rrecipe[cuisine]:
            recipe_has_chef["recipe_name"].append(rec)
            recipe_has_chef["recipe_cuisine_name"].append(cuisine)
            recipe_has_chef["chef_id"].append(id)

pd.DataFrame(recipe_has_chef).to_csv("recipe_has_chef.csv", index=False, quoting=csv.QUOTE_NONNUMERIC, quotechar='"')

In [63]:
step_data = pd.read_csv("../step.csv")

# remove ukrainian recipe steps
step_data = step_data[step_data.recipe_cuisine_name != 'Ukrainian']
#print(step_data.recipe_cuisine_name)

new_step_cuisine = {'recipe_cuisine_name':[]}
for row in step_data.itertuples(index=True, name='Pandas'):
    recipe_name = getattr(row, 'recipe_name')
    #print(recipe_name)
    new_step_cuisine['recipe_cuisine_name'].append(recipe_cuisine[recipe_name])
step_data.update(new_step_cuisine)
step_data.to_csv("step.csv", index=False, quoting=csv.QUOTE_NONNUMERIC, quotechar='"')

In [64]:
# change _has_equipment cuisines and create equipment
he = pd.read_csv("../recipe_has_equipment.csv")
he = he[he.recipe_cuisine_name != 'Ukrainian']

new_he_cuisine = {'recipe_cuisine_name':[]}

equipment = {
    "name":[],
    "manual":[],
    "image_id":[]
}

for row in he.itertuples(index=True, name='Pandas'):
    recipe_name = getattr(row, 'recipe_name')
    new_he_cuisine['recipe_cuisine_name'].append(recipe_cuisine[recipe_name])

    eq = getattr(row, 'equipment')
    if not(eq in equipment["name"]):
        equipment["name"].append(eq)
        equipment["manual"].append("used for cooking")
        equipment["image_id"].append(0)

he.update(new_he_cuisine)
he.to_csv("recipe_has_equipment.csv", index=False, quoting=csv.QUOTE_NONNUMERIC, quotechar='"')

pd.DataFrame(equipment).to_csv("equipment.csv", index=False, quoting=csv.QUOTE_NONNUMERIC, quotechar='"')

In [65]:
food_team = pd.read_csv("food_team.csv")
short_names = food_team["short_name"].to_numpy()

In [71]:
basic_ingredients = []
for row in nr_data.itertuples(index=True, name='Pandas'):
    ing = getattr(row, 'ingredient_name')
    if not(ing in basic_ingredients):
        basic_ingredients.append(ing)

food_info = {
    "fat":[],
    "protein":[],
    "carbohydrate":[],
    "total_calories":[],
    "recipe_name":[],
    "recipe_cuisine_name":[]
}

ingredient = {
    "name":[],
    "cal_per_100":[],
    "food_team_short_name":[],
    "image_id":[]
}

hi = pd.read_csv("../recipe_has_ingredient.csv")
new_hi_cuisine = {'recipe_cuisine_name':[]}
for row in hi.itertuples(index=True, name='Pandas'):
    recipe_name = getattr(row, 'recipe_name')
    new_hi_cuisine['recipe_cuisine_name'].append(recipe_cuisine[recipe_name])
    
    ing = getattr(row, 'ingredient_name')
    if not(ing in ingredient["name"]):
        ingredient["name"].append(ing)
        w = getattr(row, "ingredient_weight")
        cal_per = getattr(row, "ingredient_kcal")
        if w > 0:
            cal_per = int(cal_per/w) * 100
        ingredient["cal_per_100"].append(cal_per)
        # RANDOM FOOD TEAM SHORT NAME
        ingredient["food_team_short_name"].append(random.choice(short_names))
        ingredient["image_id"].append(0)

    if not(recipe_name in food_info["recipe_name"]):
        food_info["recipe_name"].append(recipe_name)
        food_info["recipe_cuisine_name"].append(recipe_cuisine[recipe_name])
        food_info["total_calories"].append(0)
        food_info["carbohydrate"].append(int(getattr(row, "ingredient_carbs")))
        food_info["fat"].append(random.randint(0, 50))
        food_info["protein"].append(random.randint(0, 50))
    else:
        food_info["carbohydrate"][food_info["recipe_name"].index(recipe_name)] += int(getattr(row, "ingredient_carbs"))
        
for ing in basic_ingredients:
    if not(ing in ingredient["name"]):
        print("found a fker: {0}".format(ing))
        ingredient["name"].append(ing)
        ingredient["cal_per_100"].append(20)
        ingredient["food_team_short_name"].append(random.choice(short_names))
        ingredient["image_id"].append(0)


hi.update(new_hi_cuisine)
new_hi = hi.rename(columns={'ingredient_weight': 'quantity'})
new_hi = new_hi.drop(columns=['ingredient_full_name', 'ingredient_kcal', 'ingredient_carbs'])
new_hi.quantity = new_hi.quantity.astype(int)
new_hi = new_hi.drop_duplicates(subset=['recipe_name', 'recipe_cuisine_name', 'ingredient_name'])
new_hi.to_csv("recipe_has_ingredient.csv", index=False, quoting=csv.QUOTE_NONNUMERIC, quotechar='"')

pd.DataFrame(food_info).to_csv("food_info.csv", index=False, quoting=csv.QUOTE_NONNUMERIC, quotechar='"')

pd.DataFrame(ingredient).to_csv("ingredient.csv", index=False, quoting=csv.QUOTE_NONNUMERIC, quotechar='"')

In [67]:
# Generate label and change _has_label
hl = pd.read_csv("../recipe_has_label.csv")
new_hl_cuisine = {"recipe_cuisine_name":[]}
'''
label = {
    "id":[],
    "name":[]
}

ids = []
'''
for row in hl.itertuples(index=True, name='Pandas'):
    recipe_name = getattr(row, 'recipe_name')
    new_hl_cuisine['recipe_cuisine_name'].append(recipe_cuisine[recipe_name])
    '''
    label_name = getattr(row, "label_name")
    if not(label_name in label["name"]):
        label["name"].append(label_name)
        if not label["id"]:
            label["id"].append(1)
        else:
            label["id"].append(label["id"][-1] + 1)
        ids.append(label["id"][-1])
    else:
        ids.append(label["id"][label["name"].index(label_name)])
    '''
#new_hl = hl.drop(columns=["label_name"])
#new_hl.insert(len(new_hl.columns), "label_id", ids, True)
hl.update(new_hl_cuisine)
hl.to_csv("recipe_has_label.csv", index=False, quoting=csv.QUOTE_NONNUMERIC, quotechar='"')

#pd.DataFrame(label).to_csv("label.csv", index=False)

In [68]:
# Generate meal and change _has_meal
hm = pd.read_csv("../recipe_has_meal.csv")
new_hm_cuisine = {"recipe_cuisine_name":[]}
'''
meal = {
    "id": [],
    "type": []
}

ids = []
'''
for row in hm.itertuples(index=True, name='Pandas'):
    recipe_name = getattr(row, 'recipe_name')
    new_hm_cuisine['recipe_cuisine_name'].append(recipe_cuisine[recipe_name])
    '''
    meal_type = getattr(row, "meal_type")
    if not(meal_type in meal["type"]):
        meal["type"].append(meal_type)
        if not meal["id"]:
            meal["id"].append(1)
        else:
            meal["id"].append(meal["id"][-1] + 1)
        ids.append(meal["id"][-1])
    else:
        ids.append(meal["id"][meal["type"].index(meal_type)])
    '''
#new_hm = hm.drop(columns=["meal_type"])
#new_hm.insert(len(new_hm.columns), "meal_id", ids, True)
hm.update(new_hm_cuisine)
hm.to_csv("recipe_has_meal.csv", index=False, quoting=csv.QUOTE_NONNUMERIC, quotechar='"')

#pd.DataFrame(meal).to_csv("meal.csv", index=False)

In [69]:
# Generate tip and change _has_tip
ht = pd.read_csv("../recipe_has_tip.csv")
new_ht_cuisine = {"recipe_cuisine_name":[]}

tip = {
    "id":[],
    "tip":[]
}
ids = []
for row in ht.itertuples(index=True, name='Pandas'):
    recipe_name = getattr(row, 'recipe_name')
    new_ht_cuisine['recipe_cuisine_name'].append(recipe_cuisine[recipe_name])

    tip_tip = getattr(row, "tip")
    if not tip["id"]:
        tip["id"].append(1)
    else:
        tip["id"].append(tip["id"][-1] + 1)
    ids.append(tip["id"][-1])
    tip["tip"].append(tip_tip)
    
new_ht = ht.drop(columns=["tip"])
new_ht.insert(len(new_ht.columns), "tip_id", ids, True)
new_ht.update(new_ht_cuisine)
new_ht.to_csv("recipe_has_tip.csv", index=False, quoting=csv.QUOTE_NONNUMERIC, quotechar='"')

pd.DataFrame(tip).to_csv("tip.csv", index=False, quoting=csv.QUOTE_NONNUMERIC, quotechar='"')

In [70]:
# Generate recipe_has_theme
theme_data = pd.read_csv("theme.csv")
recipe_has_theme = {
    "recipe_name": [],
    "recipe_cuisine_name": [],
    "theme_name": []
}

for key in recipe_cuisine.keys():
    recipe_has_theme["recipe_name"].append(key)
    recipe_has_theme["recipe_cuisine_name"].append(recipe_cuisine[key])
    recipe_has_theme["theme_name"].append(random.choice(theme_data["name"]))

pd.DataFrame(recipe_has_theme).to_csv("recipe_has_theme.csv", index=False, quoting=csv.QUOTE_NONNUMERIC, quotechar='"')